<a href="https://colab.research.google.com/github/dgizdevans/master/blob/main/ai_project/yolov8_unlabeled_data_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics==8.0.3

In [ ]:
import os
import json
import numpy as np
import torch
from ultralytics import YOLO
from google.colab import auth
from google.cloud import storage

In [ ]:
# Authenticate and initialize Google Cloud client
auth.authenticate_user()
project_id = "ai-group-project"
client = storage.Client(project=project_id)
bucket_name = "ai-group-project-data"
bucket = client.bucket(bucket_name)

In [ ]:
# Paths
gcs_data_path = "datasets/unlabeled_data"
gcs_model_path = "models/yolov8_training/weights/best.pt"
gcs_results_path = "results/unlabeled_data_testing"
local_data_path = "/content/unlabeled_data"
local_model_path = "/content/best.pt"
local_results_path = "/content/test_results"

In [ ]:
# Download model
print("[INFO] Downloading model from GCP...")
model_blob = bucket.blob(gcs_model_path)
model_blob.download_to_filename(local_model_path)
print("[INFO] Model downloaded successfully.")

[INFO] Downloading model from GCP...
[INFO] Model downloaded successfully.


In [ ]:
# Download test sets
print("[INFO] Downloading test sets from GCP...")
os.makedirs(local_data_path, exist_ok=True)
for test_set in range(1, 6):
    test_set_path = f"{gcs_data_path}/test_set_{test_set}"
    local_test_set_path = f"{local_data_path}/test_set_{test_set}"
    os.makedirs(local_test_set_path, exist_ok=True)

    blobs = list(bucket.list_blobs(prefix=test_set_path))
    for blob in blobs:
        local_file_path = os.path.join(local_test_set_path, os.path.basename(blob.name))
        blob.download_to_filename(local_file_path)
    print(f"[INFO] Test Set {test_set} downloaded.")

[INFO] Downloading test sets from GCP...
[INFO] Test Set 1 downloaded.
[INFO] Test Set 2 downloaded.
[INFO] Test Set 3 downloaded.
[INFO] Test Set 4 downloaded.
[INFO] Test Set 5 downloaded.


In [ ]:
# Initialize YOLO model
print("[INFO] Loading YOLO model...")
model = YOLO(local_model_path)


[INFO] Loading YOLO model...


/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:341: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(attempt_download(weight), map_location='

In [ ]:
# Process each test set
for test_set in range(1, 6):
    print(f"\n[INFO] Processing Test Set {test_set}...")
    local_test_set_path = f"{local_data_path}/test_set_{test_set}"
    local_annotated_path = f"{local_results_path}/test_set_{test_set}/annotated_images"
    local_predictions_path = f"{local_results_path}/test_set_{test_set}/predictions.json"
    local_summary_path = f"{local_results_path}/test_set_{test_set}/summary.txt"

    # Create directories
    os.makedirs(local_annotated_path, exist_ok=True)

    # Define hardcoded class names
    class_names = {
        0: "V",  # Passenger Vehicles
        1: "C",  # Cargo Vehicles
        2: "S"   # Buses
    }
    # Run predictions
    results = model.predict(
        source=local_test_set_path,
        save=True,
        conf=0.5,
        project=local_annotated_path,
        name="annotated_images"
    )
    print(f"[INFO] Predictions completed for Test Set {test_set}.")

    # Save predictions JSON
    predictions = []
    class_counts = {}
    confidence_scores = []

    for result in results:
        if isinstance(result, torch.Tensor):
            for prediction in result:
                box = prediction[:4].tolist()  # [x1, y1, x2, y2]
                conf = prediction[4].item()  # Confidence score
                cls = int(prediction[5].item())  # Class label
                class_name = class_names.get(cls, "Unknown")

                class_counts[class_name] = class_counts.get(class_name, 0) + 1
                confidence_scores.append(conf)

                predictions.append({
                    "image": "unknown",
                    "box": box,
                    "label": cls,
                    "confidence": conf
                })

    with open(local_predictions_path, "w") as f:
        json.dump(predictions, f)
    print(f"[INFO] Predictions saved for Test Set {test_set}.")

    # Save metrics
    avg_confidence = np.mean(confidence_scores) if confidence_scores else 0
    median_confidence = np.median(confidence_scores) if confidence_scores else 0
    std_confidence = np.std(confidence_scores) if confidence_scores else 0

    with open(local_summary_path, "w") as f:
        f.write(f"Metrics for Test Set {test_set}:\n")
        f.write(f"Class Distribution:\n")
        for class_name, count in class_counts.items():
            f.write(f"  {class_name}: {count}\n")
        f.write(f"\nConfidence Score Statistics:\n")
        f.write(f"  Average Confidence: {avg_confidence:.4f}\n")
        f.write(f"  Median Confidence: {median_confidence:.4f}\n")
        f.write(f"  Standard Deviation of Confidence: {std_confidence:.4f}\n")
    print(f"[INFO] Metrics saved for Test Set {test_set}.")

Ultralytics YOLOv8.0.3 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Fusing layers... 
Model summary: 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs



[INFO] Processing Test Set 1...


image 1/840 /content/unlabeled_data/test_set_1/frame_20241126_050545.jpg: 384x640 1 S, 7.9ms
image 2/840 /content/unlabeled_data/test_set_1/frame_20241126_050643.jpg: 384x640 2 Vs, 7.2ms
image 3/840 /content/unlabeled_data/test_set_1/frame_20241126_050703.jpg: 384x640 1 S, 8.0ms
image 4/840 /content/unlabeled_data/test_set_1/frame_20241126_050801.jpg: 384x640 2 Vs, 1 C, 1 S, 7.3ms
image 5/840 /content/unlabeled_data/test_set_1/frame_20241126_050820.jpg: 384x640 3 Vs, 1 S, 7.2ms
image 6/840 /content/unlabeled_data/test_set_1/frame_20241126_050957.jpg: 384x640 7.5ms
image 7/840 /content/unlabeled_data/test_set_1/frame_20241126_051036.jpg: 384x640 1 V, 8.5ms
image 8/840 /content/unlabeled_data/test_set_1/frame_20241126_051114.jpg: 384x640 2 Vs, 1 S, 7.2ms
image 9/840 /content/unlabeled_data/test_set_1/frame_20241126_051447.jpg: 384x640 3 Vs, 7.5ms
image 10/840 /content/unlabeled_data/test_set_1/frame_20241126_051623.jpg: 384x640 1 V, 7.4ms
image 11/840 /content/unlabeled_data/test_set_1/f

[INFO] Predictions completed for Test Set 1.


Ultralytics YOLOv8.0.3 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Fusing layers... 
Model summary: 168 layers, 3006233 parameters, 0 gradients
image 1/840 /content/unlabeled_data/test_set_2/frame_20241126_050446.jpg: 384x640 7.8ms
image 2/840 /content/unlabeled_data/test_set_2/frame_20241126_050507.jpg: 384x640 7.2ms
image 3/840 /content/unlabeled_data/test_set_2/frame_20241126_051016.jpg: 384x640 7.3ms


...

[INFO] Predictions completed for Test Set 4.


Ultralytics YOLOv8.0.3 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Fusing layers... 
Model summary: 168 layers, 3006233 parameters, 0 gradients
image 1/843 /content/unlabeled_data/test_set_5/frame_20241126_050722.jpg: 384x640 1 S, 7.6ms
image 2/843 /content/unlabeled_data/test_set_5/frame_20241126_050918.jpg: 384x640 7.1ms
image 3/843 /content/unlabeled_data/test_set_5/frame_20241126_051212.jpg: 384x640 1 S, 7.2ms


[INFO] Predictions saved for Test Set 4.
[INFO] Metrics saved for Test Set 4.

[INFO] Processing Test Set 5...


image 4/843 /content/unlabeled_data/test_set_5/frame_20241126_051525.jpg: 384x640 1 V, 1 S, 7.9ms
image 5/843 /content/unlabeled_data/test_set_5/frame_20241126_051800.jpg: 384x640 3 Vs, 7.3ms
image 6/843 /content/unlabeled_data/test_set_5/frame_20241126_051917.jpg: 384x640 2 Vs, 7.4ms
image 7/843 /content/unlabeled_data/test_set_5/frame_20241126_052231.jpg: 384x640 1 V, 7.3ms
image 8/843 /content/unlabeled_data/test_set_5/frame_20241126_052348.jpg: 384x640 1 S, 7.6ms
image 9/843 /content/unlabeled_data/test_set_5/frame_20241126_052446.jpg: 384x640 7.2ms
image 10/843 /content/unlabeled_data/test_set_5/frame_20241126_052917.jpg: 384x640 2 Vs, 1 C, 1 S, 8.5ms
image 11/843 /content/unlabeled_data/test_set_5/frame_20241126_052937.jpg: 384x640 1 V, 1 C, 3 Ss, 7.5ms
image 12/843 /content/unlabeled_data/test_set_5/frame_20241126_053133.jpg: 384x640 7.7ms
image 13/843 /content/unlabeled_data/test_set_5/frame_20241126_053211.jpg: 384x640 7.5ms
image 14/843 /content/unlabeled_data/test_set_5/fram

[INFO] Predictions completed for Test Set 5.
[INFO] Predictions saved for Test Set 5.
[INFO] Metrics saved for Test Set 5.


In [ ]:
# Path where metrics summaries are stored
metrics_base_path = "/content/test_results"

# Iterate through each test set
for test_set in range(1, 6):
    summary_file_path = os.path.join(metrics_base_path, f"test_set_{test_set}/summary.txt")

    print(f"\n=== Metrics for Test Set {test_set} ===")

    # Check if the summary file exists
    if not os.path.exists(summary_file_path):
        print(f"[WARNING] Metrics summary not found for Test Set {test_set}.")
        continue

    # Read and display the metrics
    try:
        with open(summary_file_path, "r") as summary_file:
            summary_content = summary_file.read()
            print(summary_content)
    except Exception as e:
        print(f"[ERROR] Unable to read metrics for Test Set {test_set}: {e}")

print("\n[INFO] Metrics display completed for all test sets.")



=== Metrics for Test Set 1 ===
Metrics for Test Set 1:
Class Distribution:
  S: 428
  V: 3564
  C: 438

Confidence Score Statistics:
  Average Confidence: 0.8067
  Median Confidence: 0.8511
  Standard Deviation of Confidence: 0.1317


=== Metrics for Test Set 2 ===
Metrics for Test Set 2:
Class Distribution:
  S: 419
  V: 3526
  C: 447

Confidence Score Statistics:
  Average Confidence: 0.8095
  Median Confidence: 0.8569
  Standard Deviation of Confidence: 0.1301


=== Metrics for Test Set 3 ===
Metrics for Test Set 3:
Class Distribution:
  C: 434
  V: 3455
  S: 425

Confidence Score Statistics:
  Average Confidence: 0.8082
  Median Confidence: 0.8560
  Standard Deviation of Confidence: 0.1321


=== Metrics for Test Set 4 ===
Metrics for Test Set 4:
Class Distribution:
  V: 3601
  S: 430
  C: 432

Confidence Score Statistics:
  Average Confidence: 0.8087
  Median Confidence: 0.8579
  Standard Deviation of Confidence: 0.1323


=== Metrics for Test Set 5 ===
Metrics for Test Set 5:
Clas

**Conclusion**:

The model shows similar performance across all five test sets.

Class Distribution:

- Passenger Vehicles (V): Dominates the predictions, with around 3450–3600 per test set.
- Cargo Vehicles (C): Shows stable predictions, with approximately 420–450 per set.
- Buses (S): Maintains a consistent count, around 420–445 per set.

Confidence Scores:

- Average Confidence: Remains stable between 0.807–0.809 across all sets.
- Median Confidence: Slightly higher, close to 0.85 in all test sets.
- Standard Deviation: Low (~0.13), indicating steady confidence levels without significant outliers.


>Summary:
The model performs well with consistent metrics and stable confidence scores across all test sets. This reliability suggests the model is ready for testing on unlabeled data or further real-world applications.

In [ ]:
# Upload results to GCP
print("\n[INFO] Uploading results to GCP...")
for test_set in range(1, 6):
    local_test_set_results_path = f"{local_results_path}/test_set_{test_set}"
    blobs = [
        os.path.join(dp, f)
        for dp, dn, filenames in os.walk(local_test_set_results_path)
        for f in filenames
    ]
    for local_file in blobs:
        relative_path = os.path.relpath(local_file, local_test_set_results_path)
        gcs_file_path = f"{gcs_results_path}/test_set_{test_set}/{relative_path}"
        blob = bucket.blob(gcs_file_path)
        blob.upload_from_filename(local_file)
    print(f"[INFO] Results uploaded for Test Set {test_set}.")
print("[INFO] All results uploaded.")


[INFO] Uploading results to GCP...
[INFO] Results uploaded for Test Set 1.
[INFO] Results uploaded for Test Set 2.
[INFO] Results uploaded for Test Set 3.
[INFO] Results uploaded for Test Set 4.
[INFO] Results uploaded for Test Set 5.
[INFO] All results uploaded.
